In [ ]:
import pandas as pd
import tqdm
import os

In [2]:
path = os.getenv('HOME')
df = pd.read_csv(path +'/data/raw/dataset_full_v1.csv')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2994182 entries, 0 to 2994181
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   text    object
 1   dial    object
 2   reg     object
 3   pair    object
dtypes: object(4)
memory usage: 91.4+ MB


In [4]:
df.head()

,text,dial,reg,pair
0,그 부주를 하기에는 조금 그렇지 (않아?),그 부주를 하기에는 조금 그렇지 (안?),jj,(안?)/(않아?)
1,그리고 거기서 밥을 먹을지도 (모르겠어.),그리고 거기서 밥을 먹을지도 (모르겐.),jj,(모르겐.)/(모르겠어.)
2,왜냐하면은 밥이 안 나온다고 (들었어.),왜냐하면은 밥이 안 나온다고 (들언.),jj,(들언.)/(들었어.)
3,토요일 날에 토요일날에 잔치를 하고 일요일 날에 식을 (올린대.),토요일 날에 토요일날에 잔치를 하고 일요일 날에 식을 (올린댄.),jj,(올린댄.)/(올린대.)
4,그럼 어떻게 혼자 아 (혼자래.),그럼 어떻게 혼자 아 (혼자랜.),jj,(혼자랜.)/(혼자래.)


In [5]:
import re

In [6]:
#data cleaning

df['text']=df['text'].apply(lambda x : re.sub('[\(\)]','',str(x))) # remove parenthesis
df['text']=df['text'].apply(lambda x : re.sub('(\&+[a-zA-Zㄱ-힣\&]+)','',str(x))) # remove words like &&name, &company name, etc.
df['text']=df['text'].apply(lambda x : re.sub('(\@+[ㄱ-힣\@]+)','',str(x))) # remove words like &&name, &company name, etc.
df['text']=df['text'].apply(lambda x : re.sub('(x+[a-zㄱ-힣]+)','',str(x))) # remove words like xx하면, xxxx, etc
df['text']=df['text'].apply(lambda x : re.sub('(\-+[a-zㄱ-힣\-]+)','',str(x))) # remove words like --하면, -시사- etc
df['text']=df['text'].apply(lambda x : re.sub(r'(.+)\1{2,}',r'\1',str(x))) # remove repeating words
df['text']=df['text'].apply(lambda x : re.sub('([ㄱ-힣]+)\~$',r'\1.',str(x))) # sub words ending in ~ to . ex 그럴거야~ 그럴거야.
df['text']=df['text'].apply(lambda x : re.sub('([ㄱ-힣]+)\~','',str(x))) # remove expressions like 어~ 아~ 으~ 오~

df['dial']=df['dial'].apply(lambda x : re.sub('[\(\)]','',str(x)))
df['dial']=df['dial'].apply(lambda x : re.sub('(\&+[a-zA-Zㄱ-힣\&]+)','',str(x)))
df['dial']=df['dial'].apply(lambda x : re.sub('(\@+[ㄱ-힣\@]+)','',str(x))) # remove words like @이름,@주소,@상호명, etc.
df['dial']=df['dial'].apply(lambda x : re.sub('(x+[a-zㄱ-힣]+)','',str(x))) # remove words like xx하면, xxxx, etc
df['dial']=df['dial'].apply(lambda x : re.sub('(\-+[a-zㄱ-힣\-\&]+)','',str(x))) # remove words like --하면, -시사- etc
df['dial']=df['dial'].apply(lambda x : re.sub(r'(.+)\1{2,}',r'\1',str(x))) # remove repeating words
df['dial']=df['dial'].apply(lambda x : re.sub('([ㄱ-힣]+)\~$',r'\1.',str(x))) # sub words ending in ~ to . ex 그럴거야~ 그럴거야.
df['dial']=df['dial'].apply(lambda x : re.sub('([ㄱ-힣]+)\~','',str(x))) # remove expressions like 어~ 아~ 으~ 오~

#remove from training

df.loc[df['text'].str.contains('/'),'rm'] = 1 #containing mislabelled data
df['len'] = df['text'].apply(lambda x: len(x))
df.loc[df['len'] < 2,'rm'] = 1 #length is less than 2
df.drop_duplicates(subset='dial', inplace=True) # drop duplicated senteces in dial
clean_df = df.loc[df['rm']!=1,['text','dial','reg','pair']]

In [7]:
clean_df.head()

,text,dial,reg,pair
0,그 부주를 하기에는 조금 그렇지 않아?,그 부주를 하기에는 조금 그렇지 안?,jj,(안?)/(않아?)
1,그리고 거기서 밥을 먹을지도 모르겠어.,그리고 거기서 밥을 먹을지도 모르겐.,jj,(모르겐.)/(모르겠어.)
2,왜냐하면은 밥이 안 나온다고 들었어.,왜냐하면은 밥이 안 나온다고 들언.,jj,(들언.)/(들었어.)
3,토요일 날에 토요일날에 잔치를 하고 일요일 날에 식을 올린대.,토요일 날에 토요일날에 잔치를 하고 일요일 날에 식을 올린댄.,jj,(올린댄.)/(올린대.)
4,그럼 어떻게 혼자 아 혼자래.,그럼 어떻게 혼자 아 혼자랜.,jj,(혼자랜.)/(혼자래.)


In [8]:
import transformers

In [33]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, pipeline
ckpt = 'circulus/kobart-trans-ko-en-v2'
tokenizer = AutoTokenizer.from_pretrained(ckpt)
model = AutoModelForSeq2SeqLM.from_pretrained(ckpt)
translator = pipeline("translation", model=ckpt, max_length=60)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.


In [40]:
text = clean_df.text.values.tolist()
len(text)

35339

In [41]:
from torch.utils.data import DataLoader

dataloader = DataLoader(text,  batch_size=512)

In [64]:
translation_file = input() # translation file name
outputs = []
for n,i in enumerate(tqdm.tqdm(dataloader)) :
    inputs = tokenizer(i, return_tensors="pt", padding=True).input_ids
    out = model.generate(inputs,num_beams=3, max_new_tokens=60) # do_sample=True, top_k=30, top_p=0.95,
    for j in out :
        result = tokenizer.decode(j, skip_special_tokens=True)
        outputs.append(result)
    
    if n % 100 == 0 :
        with open(translation_file,'w') as f :
            for sen in outputs :
                f.write(sen+'\n')
                
with open(translation_file,'w') as f :
    for sen in outputs :
        f.write(sen+'\n')

100%|███████████████████████████████████████████| 70/70 [46:05<00:00, 39.51s/it]


In [82]:
with open(translation_file,'r') as f :
    raw = f.read().splitlines()

reg
jj    1114693
kw     497897
gs     332121
jd     287385
cc     274155
Name: count, dtype: int64

In [80]:
translated_train_data = input() #file name for translated trained data
clean_df['eng'] = raw
clean_df.to_csv(f'../../data/processed/{translated_train_data}.csv')